In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import tensorflow as tf
import cv2
import numpy as np
from tensorflow.keras.applications import MobileNetV3Small, MobileNetV3Large
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

#MobileNetV3

In [ ]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import MobileNetV3Small, MobileNetV3Large
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Load your dataset
trainingData = np.load('/content/drive/MyDrive/myColab/myDataset/trainingData.npy')
trainingLabels = np.load('/content/drive/MyDrive/myColab/myDataset/trainingLabels.npy')

validationData = np.load('/content/drive/MyDrive/myColab/myDataset/validationData.npy')
validationLabels = np.load('/content/drive/MyDrive/myColab/myDataset/validationLabels.npy')

testingData = np.load('/content/drive/MyDrive/myColab/myDataset/testingData.npy')
testingLabels = np.load('/content/drive/MyDrive/myColab/myDataset/testingLabels.npy')

# Normalize pixel values to be between 0 and 1
trainingData = trainingData / 255.0
validationData = validationData / 255.0
testingData = testingData / 255.0

# Create the MobileNetV3 model with pre-trained weights (exclude top layers)
# base_model = MobileNetV3Small(weights='imagenet', include_top=False, input_shape=(48, 48, 3))
base_model = MobileNetV3Large(weights='imagenet', include_top=False, input_shape=(48, 48, 3))

# Freeze the layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

# Add your custom layers on top for emotion classification
model = keras.Sequential([
    layers.Input(shape=(48, 48, 1)),
    layers.Conv2D(3, (3, 3), padding='same'),  # Convert single-channel to 3-channel
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    # layers.Dropout(0.2),
    layers.Dense(7, activation='softmax')  # 7 output classes for emotions
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Callbacks for saving the best model and early stopping
checkpoint = ModelCheckpoint('/content/drive/MyDrive/myColab/best_model_mobilenetv3.h5',
                             monitor='val_accuracy',
                             save_best_only=True,
                             mode='max',
                             verbose=1)

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(trainingData, trainingLabels,
                    epochs=20,
                    batch_size=32,
                    validation_data=(validationData, validationLabels),
                    callbacks=[checkpoint, early_stopping])

# Evaluate on the test set
test_loss, test_acc = model.evaluate(testingData, testingLabels)
print(f'Test Accuracy: {test_acc * 100:.2f}%')


Epoch 1/20
783/785 [============================>.] - ETA: 0s - loss: 1.8399 - accuracy: 0.2340
Epoch 1: val_accuracy improved from -inf to 0.24802, saving model to /content/drive/MyDrive/myColab/best_model_mobilenetv3.h5
785/785 [==============================] - 19s 19ms/step - loss: 1.8397 - accuracy: 0.2342 - val_loss: 1.8114 - val_accuracy: 0.2480
Epoch 2/20
785/785 [==============================] - ETA: 0s - loss: 1.8099 - accuracy: 0.2516
Epoch 2: val_accuracy did not improve from 0.24802
785/785 [==============================] - 13s 17ms/step - loss: 1.8099 - accuracy: 0.2516 - val_loss: 1.8102 - val_accuracy: 0.2480
Epoch 3/20
783/785 [============================>.] - ETA: 0s - loss: 1.8057 - accuracy: 0.2518
Epoch 3: val_accuracy improved from 0.24802 to 0.24830, saving model to /content/drive/MyDrive/myColab/best_model_mobilenetv3.h5
785/785 [==============================] - 12s 16ms/step - loss: 1.8060 - accuracy: 0.2515 - val_loss: 1.8023 - val_accuracy: 0.2483
Epoch 4

#VGG16

In [ ]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Load your dataset
trainingData = np.load('/content/drive/MyDrive/myColab/myDataset/trainingData.npy')
trainingLabels = np.load('/content/drive/MyDrive/myColab/myDataset/trainingLabels.npy')

validationData = np.load('/content/drive/MyDrive/myColab/myDataset/validationData.npy')
validationLabels = np.load('/content/drive/MyDrive/myColab/myDataset/validationLabels.npy')

testingData = np.load('/content/drive/MyDrive/myColab/myDataset/testingData.npy')
testingLabels = np.load('/content/drive/MyDrive/myColab/myDataset/testingLabels.npy')

# Normalize pixel values to be between 0 and 1
trainingData = trainingData / 255.0
validationData = validationData / 255.0
testingData = testingData / 255.0

# Create the VGG16 model with pre-trained weights (exclude top layers)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(48, 48, 3))

# Freeze the layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

# Add your custom layers on top for emotion classification
model = keras.Sequential([
    layers.Input(shape=(48, 48, 1)),
    layers.Conv2D(3, (3, 3), padding='same'),  # Convert single-channel to 3-channel
    base_model,
    layers.Flatten(),
    # layers.Dense(256, activation='relu'),
    # layers.Dropout(0.2),
    # layers.Dropout(0.3),
    layers.Dense(7, activation='softmax')  # 7 output classes for emotions
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Callbacks for saving the best model and early stopping
checkpoint = ModelCheckpoint('/content/drive/MyDrive/myColab/best_model.h5',
                             monitor='val_accuracy',
                             save_best_only=True,
                             mode='max',
                             verbose=1)

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(trainingData, trainingLabels,
                    epochs=20,
                    batch_size=32,
                    validation_data=(validationData, validationLabels),
                    callbacks=[checkpoint, early_stopping])

# Evaluate on the test set
test_loss, test_acc = model.evaluate(testingData, testingLabels)
print(f'Test Accuracy: {test_acc * 100:.2f}%')


Epoch 1/20
784/785 [============================>.] - ETA: 0s - loss: 1.8385 - accuracy: 0.2344
Epoch 1: val_accuracy improved from -inf to 0.26022, saving model to /content/drive/MyDrive/myColab/best_model.h5
785/785 [==============================] - 22s 27ms/step - loss: 1.8385 - accuracy: 0.2343 - val_loss: 1.7812 - val_accuracy: 0.2602
Epoch 2/20
785/785 [==============================] - ETA: 0s - loss: 1.7555 - accuracy: 0.2756
Epoch 2: val_accuracy improved from 0.26022 to 0.28740, saving model to /content/drive/MyDrive/myColab/best_model.h5
785/785 [==============================] - 23s 30ms/step - loss: 1.7555 - accuracy: 0.2756 - val_loss: 1.7456 - val_accuracy: 0.2874
Epoch 3/20
783/785 [============================>.] - ETA: 0s - loss: 1.7144 - accuracy: 0.3020
Epoch 3: val_accuracy improved from 0.28740 to 0.30514, saving model to /content/drive/MyDrive/myColab/best_model.h5
785/785 [==============================] - 21s 27ms/step - loss: 1.7145 - accuracy: 0.3018 - val_l

#LeNet5

In [ ]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Model

# Load your dataset
trainingData = np.load('/content/drive/MyDrive/myColab/myDataset/trainingData.npy')
trainingLabels = np.load('/content/drive/MyDrive/myColab/myDataset/trainingLabels.npy')

validationData = np.load('/content/drive/MyDrive/myColab/myDataset/validationData.npy')
validationLabels = np.load('/content/drive/MyDrive/myColab/myDataset/validationLabels.npy')

testingData = np.load('/content/drive/MyDrive/myColab/myDataset/testingData.npy')
testingLabels = np.load('/content/drive/MyDrive/myColab/myDataset/testingLabels.npy')

# Normalize pixel values to be between 0 and 1
trainingData = trainingData / 255.0
validationData = validationData / 255.0
testingData = testingData / 255.0

# Create LeNet-5 model
model = keras.Sequential([
    layers.Input(shape=(48, 48, 1)),
    # layers.Conv2D(6, (5, 5), activation='relu', padding='valid'),
    # layers.MaxPooling2D((2, 2)),
    # layers.Conv2D(16, (5, 5), activation='relu', padding='valid'),
    # layers.MaxPooling2D((2, 2)),
    # layers.Flatten(),
    # layers.Dense(120, activation='relu'),
    # layers.Dropout(0.2),
    # layers.Dense(84, activation='relu'),
    # layers.Dropout(0.3),
    layers.Dense(7, activation='softmax')  # 7 output classes for emotions
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Callbacks for saving the best model and early stopping
checkpoint = ModelCheckpoint('/content/drive/MyDrive/myColab/best_lenet_model.h5',
                             monitor='val_accuracy',
                             save_best_only=True,
                             mode='max',
                             verbose=1)

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(trainingData, trainingLabels,
                    epochs=20,
                    batch_size=32,
                    validation_data=(validationData, validationLabels),
                    callbacks=[checkpoint, early_stopping])

# Evaluate on the test set
test_loss, test_acc = model.evaluate(testingData, testingLabels)
print(f'Test Accuracy: {test_acc * 100:.2f}%')
